In [92]:
_ARREST_KEY = "ARREST_KEY"

_ARREST_BORO = "ARREST_BORO"
_JURISDICTION_CODE = "JURISDICTION_CODE"
_OFFENSE_LEVEL = "LAW_CAT_CD"

_ARREST_DATE = "ARREST_DATE"
_ARREST_MONTH = "ARREST_MONTH" 

_ARREST_PRECINCT = "ARREST_PRECINCT"

_PERPETRATOR_RACE = "PERP_RACE"
_PERPETRATOR_SEX = "PERP_SEX"
_PERPETRATOR_AGE_GROUP="AGE_GROUP"

_INTERNAL_CLASSIFICATION="PD_CD"
_INTERNAL_CLASSIFICATION_DESCRIPTION="PD_DESC"
_THREE_DIGIT_INTERNAL_CLASSIFICATION="KY_CD"

_LAW_CODE="LAW_CODE"

_GEOGRAPHICAL_POSITION = "New Georeferenced Column"

_NYS_X_COORD = "X_COORD_CD"
_NYS_Y_COORD = "Y_COORD_CD"

_LATITUDE='Latitude'
_LONGITUDE='Longitude'


In [93]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [94]:
df = pd.read_csv('./cleaned_data.csv')
df.head()


,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,Latitude,Longitude,ARREST_MONTH
0,FELONY ASSAULT,PL-1211200,F,Q,105,0,40.737043,-73.735514,January
1,FELONY ASSAULT,PL-1200502,F,B,48,0,40.855109,-73.892818,March
2,FELONY ASSAULT,PL-1200512,F,S,121,0,40.628967,-74.163275,May
3,FELONY ASSAULT,PL-1211200,F,Q,100,0,40.591980,-73.800066,June
4,RAPE,PL-1302503,F,M,14,0,40.753533,-73.994537,January


In [95]:
df[_ARREST_PRECINCT] = df[_ARREST_PRECINCT].astype('object')


In [96]:
df_num= df.select_dtypes(exclude='object')
df_obj= df.select_dtypes(include='object')

df_num.info()
df_obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188682 entries, 0 to 188681
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   JURISDICTION_CODE  188682 non-null  int64  
 1   Latitude           188682 non-null  float64
 2   Longitude          188682 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 4.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188682 entries, 0 to 188681
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   OFNS_DESC        188682 non-null  object
 1   LAW_CODE         188682 non-null  object
 2   LAW_CAT_CD       188682 non-null  object
 3   ARREST_BORO      188682 non-null  object
 4   ARREST_PRECINCT  188682 non-null  object
 5   ARREST_MONTH     188682 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [97]:
df_obj[_OFFENSE_LEVEL].unique()
df_obj=df_obj.drop(columns=["OFNS_DESC","LAW_CODE"])# drop some columns to deal with out of mem error

In [98]:
df_obj.head()

,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,ARREST_MONTH
0,F,Q,105,January
1,F,B,48,March
2,F,S,121,May
3,F,Q,100,June
4,F,M,14,January


In [ ]:
df_obj= pd.get_dummies(df_obj, drop_first=True)

target = (df_obj['LAW_CAT_CD_M'])
df_obj = df_obj.drop(columns=['LAW_CAT_CD_M'],axis=1)
df_obj

In [ ]:
attr = pd.concat([df_obj,df_num],axis=1)


print(attr.shape[1])
attr.head()

96


Index(['LAW_CAT_CD_F', 'LAW_CAT_CD_V', 'ARREST_BORO_B', 'ARREST_BORO_K',
       'ARREST_BORO_M', 'ARREST_BORO_Q', 'ARREST_BORO_S', 'ARREST_PRECINCT_1',
       'ARREST_PRECINCT_5', 'ARREST_PRECINCT_6', 'ARREST_PRECINCT_7',
       'ARREST_PRECINCT_9', 'ARREST_PRECINCT_10', 'ARREST_PRECINCT_13',
       'ARREST_PRECINCT_14', 'ARREST_PRECINCT_17', 'ARREST_PRECINCT_18',
       'ARREST_PRECINCT_19', 'ARREST_PRECINCT_20', 'ARREST_PRECINCT_22',
       'ARREST_PRECINCT_23', 'ARREST_PRECINCT_24', 'ARREST_PRECINCT_25',
       'ARREST_PRECINCT_26', 'ARREST_PRECINCT_28', 'ARREST_PRECINCT_30',
       'ARREST_PRECINCT_32', 'ARREST_PRECINCT_33', 'ARREST_PRECINCT_34',
       'ARREST_PRECINCT_40', 'ARREST_PRECINCT_41', 'ARREST_PRECINCT_42',
       'ARREST_PRECINCT_43', 'ARREST_PRECINCT_44', 'ARREST_PRECINCT_45',
       'ARREST_PRECINCT_46', 'ARREST_PRECINCT_47', 'ARREST_PRECINCT_48',
       'ARREST_PRECINCT_49', 'ARREST_PRECINCT_50', 'ARREST_PRECINCT_52',
       'ARREST_PRECINCT_60', 'ARREST_PRECINCT_61'

In [101]:
target

0         False
1         False
2         False
3         False
4         False
          ...  
188677     True
188678     True
188679    False
188680     True
188681    False
Name: LAW_CAT_CD_M, Length: 188682, dtype: bool

In [102]:
from sklearn.model_selection import train_test_split

# Split the data into training and test
attr_train, attr_test, target_train, target_test = train_test_split(attr, target, test_size=0.40, random_state=5)

In [103]:
from sklearn.compose import make_column_selector

cat_selector = make_column_selector(dtype_include=bool)
num_selector = make_column_selector(dtype_include=np.number)
num_selector(attr_train)

['JURISDICTION_CODE', 'Latitude', 'Longitude']

In [104]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(
            handle_unknown="infrequent_if_exist"
            ), df_obj.columns.tolist()),
        ('num', StandardScaler(), df_num.columns.tolist()),
    ],     remainder='passthrough'
)

In [105]:
#CART
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline


model = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',DecisionTreeClassifier())
])



In [106]:

model.fit(attr_train,target_train)


ValueError: A given column is not a column of the dataframe

In [ ]:
target_pred = model.predict(attr_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(target_test,target_pred))
print(classification_report(target_test,target_pred))

[[32917     0]
 [    0 42556]]
              precision    recall  f1-score   support

       False       1.00      1.00      1.00     32917
        True       1.00      1.00      1.00     42556

    accuracy                           1.00     75473
   macro avg       1.00      1.00      1.00     75473
weighted avg       1.00      1.00      1.00     75473



In [ ]:
attr.columns

Index(['LAW_CAT_CD_M', 'ARREST_BORO_K', 'ARREST_BORO_M', 'ARREST_BORO_Q',
       'ARREST_BORO_S', 'ARREST_PRECINCT_5', 'ARREST_PRECINCT_6',
       'ARREST_PRECINCT_7', 'ARREST_PRECINCT_9', 'ARREST_PRECINCT_10',
       'ARREST_PRECINCT_13', 'ARREST_PRECINCT_14', 'ARREST_PRECINCT_17',
       'ARREST_PRECINCT_18', 'ARREST_PRECINCT_19', 'ARREST_PRECINCT_20',
       'ARREST_PRECINCT_22', 'ARREST_PRECINCT_23', 'ARREST_PRECINCT_24',
       'ARREST_PRECINCT_25', 'ARREST_PRECINCT_26', 'ARREST_PRECINCT_28',
       'ARREST_PRECINCT_30', 'ARREST_PRECINCT_32', 'ARREST_PRECINCT_33',
       'ARREST_PRECINCT_34', 'ARREST_PRECINCT_40', 'ARREST_PRECINCT_41',
       'ARREST_PRECINCT_42', 'ARREST_PRECINCT_43', 'ARREST_PRECINCT_44',
       'ARREST_PRECINCT_45', 'ARREST_PRECINCT_46', 'ARREST_PRECINCT_47',
       'ARREST_PRECINCT_48', 'ARREST_PRECINCT_49', 'ARREST_PRECINCT_50',
       'ARREST_PRECINCT_52', 'ARREST_PRECINCT_60', 'ARREST_PRECINCT_61',
       'ARREST_PRECINCT_62', 'ARREST_PRECINCT_63', 'ARREST_

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

classifier = model.named_steps['classifier']
plt.figure(figsize=(50,30), dpi=250)
plot_tree(classifier,fontsize=20,filled=True,feature_names=attr.columns)

[Text(0.5, 0.75, 'LAW_CAT_CD_M <= 0.5\ngini = 0.492\nsamples = 113209\nvalue = [49537.0, 63672.0]'),
 Text(0.25, 0.25, 'gini = 0.0\nsamples = 63672\nvalue = [0, 63672]'),
 Text(0.375, 0.5, 'True  '),
 Text(0.75, 0.25, 'gini = 0.0\nsamples = 49537\nvalue = [49537, 0]'),
 Text(0.625, 0.5, '  False')]